# Late blight prediction for the Columbia Basin of Washington

# Date: 4/7/2021

# Authors:
- Original models were developed by Dennis Johnson and colleagues:
    - Johnson DA, Alldredge JR, and Vakoch DL. 1996. Potato late blight forecasting models for the semiarid environment of south-central Washington. Phytopathology 86:480-484. https://www.apsnet.org/publications/phytopathology/backissues/Documents/1996Articles/Phyto86n05_480.PDF
    - Johnson DA, Alldredge JR, and Hamm PB. 1998. Expansion of potato late blight forecasting models for the Columbia Basin of Washington and Oregon. Plant Dis. 82:642-645. https://apsjournals.apsnet.org/doi/pdfplus/10.1094/PDIS.1998.82.6.642
    - Johnson DA, Cummings TF, Abi Ghanem R, and Alldredge JR. 2009. Association of solar irradiance and days of precipitation with incidence of potato late blight in the semiarid environment of the Columbia Basin. Plant Dis. 93:272-280. https://apsjournals.apsnet.org/doi/pdfplus/10.1094/PDIS-93-3-0272
    - Johnson DA, and Cummings TF. 2016. In-canopy environment of sprinkler irrigated potato fields as a factor for late blight management in the semiarid environment of the Columbia Basin. Am J. Potato Res: 93:239-252 https://link.springer.com/article/10.1007/s12230-016-9500-1

**Script written by: David Linnard Wheeler and Sudha GC Upadhaya**

       

# Objectives

**Predict late blight epidemics in the Columbia Basin of Washington State**

# Risk

**Risk of late blight is the product of 5 factors**

**Risk** = **A** (phenology) $\cdot$ **B** (occurrence of late blight in field or adjacent fields) $\cdot$ **C** (probability of late blight occurrence in the Columbia Basin (PROB)) $\cdot$ **D** (date) $\cdot$ **E** (number of rainy days expected in the next 7 days)

### A (phenology)

**Source of information: growers**

| Factor level | Factor value |
| --- | --- |
| Pre-emergence | 0 |
| Emergence to before plant closure between rows | 0 |
| Plant closure between rows (foliage touching between adjacent rows) | 1 |
|Post row closure to harvest | 1 |

### B (occurrence of late blight in field or adjacent fields within 5 mile radius)

**Source of information: growers**

| Factor level | Factor value |
| --- | --- |
| No | 0 |
| Yes | 30 |

### C (probability of late blight occurrence in the Columbia Basin (PROB))

#### For the logistic regression models, the probability, $P$, of an outbreak is estimated:
$$
P = \frac{1}{1 + exp^{lf}}
$$

> - if 
$$  
P
\begin{cases}
  \geq 0.5 \rightarrow \text{ outbreak year} \\    
  < 0.5 \rightarrow \text{ non-outbreak year}
\end{cases}
$$

| Factor level | Factor value |
| --- | --- |
| < 50% | 0 |
| $\geq$ 50% | 4 |

### D (date)

**Source of information: internet**

| Factor level | Factor value |
| --- | --- |
| December - April | 0 |
| May, July - November | 1 |
| June | 3J |

### E (number of rainy days expected in the next 7 days) (> 30% probability)

**Source of information: Rain forecasts from Fox Weather LLC. A rain probability of 30% has arbitrarily been used to indicate rain**

| Factor level | Factor value |
| --- | --- |
| 0 rainy days expected | 1 |
| 1 rainy days expected | 2 |
| 2 rainy days expected| 4 |
| 3 or more rainy days expected| 5 |

In [1]:
import os
import pandas as pd
import numpy as np
import datetime

**A**

In [2]:
while True:
    try:
    # Request user input
        A = input("What is the phenology of the potato field?\n \
                 A) Pre-emergence\n \
                 B) Emergence to before plant closure between rows\n \
                 C) Plant closure between rows (foliage touching between adjacent rows)\n \
                 D) Post row closure to harvest\n")
        A = A.upper()
    # If Risk Factor (RF) A is option "A" or option "B"
        if (A == "A") or (A == "B"):
        # Set RF to 0
            A = 0
            break
    # Else, RF is "C" or "D" and
        elif (A == "C") or (A == "D"):
        # Set it to 1
            A = 1
            break
    #Else, RF is no of the above return warning
        else: 
            print('Please, Enter the valid option!!')  
    except ValueError:
        # Break
        break

What is the phenology of the potato field?
                  A) Pre-emergence
                  B) Emergence to before plant closure between rows
                  C) Plant closure between rows (foliage touching between adjacent rows)
                  D) Post row closure to harvest
 a


**B**

In [169]:
while True:
    try:
    # Request user input
        B = input("Is late blight present within a 5 mile radius?\n \
                 A) No\n \
                 B) Yes\n")
        B = B.upper()
    # If Risk Factor (RF) B is option "A"
        if (B == "A") :
        # Set RF to 0
            B = 0
            break
    # Else, RF is "B" and
        elif (B == "B"):
        # Set it to 30
            B = 30
            break
    #Else, RF is no of the above return warning
        else: 
            print('Please, Enter the valid option!!')
    except ValueError:
        # Break
        break

Is late blight present within a 5 mile radius?
                  A) No
                  B) Yes
 a


**C**

#### Weather April and May

In [3]:
os.chdir('C:/Users/theox/Desktop/Projects/LateBlight/WeatherForecasts/2022/Monthly')
df_othello = pd.read_csv('Othello_AM.csv')
df_prosser = pd.read_csv('Prosser_AM.csv')
df_tricities = pd.read_csv('Tricities_AM.csv')

In [4]:
df_othello['Station'] = 'Othello'
df_prosser['Station'] = 'Prosser'
df_tricities['Station'] = 'Tricities'

In [5]:
df_AM = pd.concat([df_othello, df_prosser, df_tricities])
df_AM.head()

,Date,Date.1,Min°F,Avg°F,Max°F,Avg1.5m DP°F,Avg1.5m RH%,2 in.°F,Min°F.1,Avg°F.1,...,Avg 8in.SWPkPa,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station,Avg1.5m LWu.,AvgDir,Avg Speedmph,2m MaxGustmph
0,2022/04/01,1,27.1,43.8,56.2,28.9,58.3,51.8,49.1,52.2,...,-469.6,0.00,19.88,0.11,0.14,Othello,NaN,NaN,NaN,NaN
1,2022/04/02,2,38.0,48.8,60.0,29.6,52.9,53.3,50.9,53.0,...,-652.1,0.04,21.54,0.16,0.24,Othello,NaN,NaN,NaN,NaN
2,2022/04/03,3,33.0,46.7,60.2,31.8,58.1,52.2,49.8,52.3,...,-840.3,0.00,16.32,0.12,0.18,Othello,NaN,NaN,NaN,NaN
3,2022/04/04,4,36.9,47.9,56.1,31.1,55.2,52.5,51.6,53.0,...,-1011.4,0.05,19.57,0.15,0.23,Othello,NaN,NaN,NaN,NaN
4,2022/04/05,5,30.3,42.9,55.8,22.4,48.3,50.2,47.8,50.7,...,-1219.5,0.00,22.55,0.16,0.23,Othello,NaN,NaN,NaN,NaN


In [6]:
df_AM = df_AM[['Date', 'TotPrecin','Station']]#subset only three columns
df_AM['Month']= pd.DatetimeIndex(df_AM['Date']).month#create a new column for month using date 
df_AM.head()

,Date,TotPrecin,Station,Month
0,2022/04/01,0.00,Othello,4
1,2022/04/02,0.04,Othello,4
2,2022/04/03,0.00,Othello,4
3,2022/04/04,0.05,Othello,4
4,2022/04/05,0.00,Othello,4


In [7]:
grouped_AM = df_AM[df_AM['TotPrecin'].astype(bool)].groupby(['Station', 'Month']).size().reset_index(name = 'RainyDays')#calculate rainy days
grouped_AM

,Station,Month,RainyDays
0,Othello,4,13
1,Othello,5,11
2,Prosser,4,9
3,Prosser,5,12
4,Tricities,4,10
5,Tricities,5,14


In [8]:
RD_May_Othello = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Othello') & (grouped_AM['Month'] ==5)])
RD_May_Prosser = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Prosser') & (grouped_AM['Month'] ==5)])
RD_May_Tricities = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Tricities') & (grouped_AM['Month'] ==5)])
RD_Apr_Othello = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Othello') & (grouped_AM['Month'] ==4)])
RD_Apr_Prosser = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Prosser') & (grouped_AM['Month'] ==4)])
RD_Apr_Tricities = np.array(grouped_AM['RainyDays'][(grouped_AM['Station']=='Tricities') & (grouped_AM['Month'] ==4)])

#### Weather July and August

In [9]:
os.chdir('C:/Users/theox/Desktop/Projects/LateBlight/WeatherForecasts/2022/Monthly')
df1_othello = pd.read_csv('Othello_JA.csv')
df1_prosser = pd.read_csv('Prosser_JA.csv')
df1_tricities = pd.read_csv('Tricities_JA.csv')

In [10]:
df1_othello['Station'] = 'Othello'
df1_prosser['Station'] = 'Prosser'
df1_tricities['Station'] = 'Tricities'

In [11]:
df_JA = pd.concat([df1_othello, df1_prosser, df1_tricities])
df_JA.head()

,Date,Date.1,Min°F,Avg°F,Max°F,Avg1.5m DP°F,Avg1.5m RH%,2 in.°F,Min°F.1,Avg°F.1,...,Avg 8in.SWPkPa,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station,Avg1.5m LWu.,AvgDir,Avg Speedmph,2m MaxGustmph
0,2022/07/01,1,54.3,72.2,87.4,50.5,50.4,83.6,77.4,81.4,...,-764.2,0.00,29.92,0.25,0.32,Othello,NaN,NaN,NaN,NaN
1,2022/07/02,2,52.5,70.6,85.7,53.1,56.9,83.3,78.4,81.7,...,-723.6,0.17,24.98,0.23,0.30,Othello,NaN,NaN,NaN,NaN
2,2022/07/03,3,55.1,64.2,77.5,54.4,72.6,77.2,76.1,78.2,...,-756.6,0.07,20.63,0.18,0.24,Othello,NaN,NaN,NaN,NaN
3,2022/07/04,4,53.9,63.5,77.2,49.2,63.2,74.8,71.8,75.3,...,-823.6,0.00,22.09,0.20,0.27,Othello,NaN,NaN,NaN,NaN
4,2022/07/05,5,49.9,66.1,81.8,53.1,66.8,77.1,71.6,75.8,...,-823.2,0.00,26.66,0.22,0.27,Othello,NaN,NaN,NaN,NaN


In [12]:
df_JA = df_JA[['Date', 'TotPrecin','Station']]#subset only three columns
df_JA['Month']= pd.DatetimeIndex(df_JA['Date']).month#create a new column for month using date 
df_JA.head()

,Date,TotPrecin,Station,Month
0,2022/07/01,0.00,Othello,7
1,2022/07/02,0.17,Othello,7
2,2022/07/03,0.07,Othello,7
3,2022/07/04,0.00,Othello,7
4,2022/07/05,0.00,Othello,7


In [16]:
grouped_JA = df_JA[df_JA['TotPrecin'].astype(bool)].groupby(['Station', 'Month']).size().reset_index(name = 'RainyDays')#calculate rainy days
grouped_JA

,Station,Month,RainyDays
0,Othello,7,4
1,Prosser,7,1
2,Tricities,7,2


In [17]:
RD_Jul_Othello = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Othello') & (grouped_JA['Month'] ==7)])
RD_Jul_Prosser = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Prosser') & (grouped_JA['Month'] ==7)])
RD_Jul_Tricities = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Tricities') & (grouped_JA['Month'] ==7)])
#RD_Aug_Othello = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Othello') & (grouped_JA['Month'] ==8)])
#RD_Aug_Prosser = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Prosser') & (grouped_JA['Month'] ==8)])
#RD_Aug_Tricities = np.array(grouped_JA['RainyDays'][(grouped_JA['Station']=='Tricities') & (grouped_JA['Month'] ==8)])

In [18]:
RD_Aug_Othello =0
RD_Aug_Prosser = 0
RD_Aug_Tricities= 0

### Forecast July and August (Monthly)

In [19]:
df_forecast = pd.read_excel('8-18_9-22.xlsx', engine='openpyxl')
print(df_forecast.shape)
df_forecast = df_forecast.iloc[4:39]
df_forecast.head()

(46, 11)


,One Month Forecast of Daily Rainfall,Unnamed: 1,Unnamed: 2,,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122,DayNmbr
5,2022-08-18 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,1
6,2022-08-19 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,2
7,2022-08-20 00:00:00,TriCities,0.01,NaN,Prosser,NaN,0.02,NaN,Othello,0.01,3
8,2022-08-21 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,4


In [20]:
df_forecast.columns = df_forecast.iloc[0]#rename column
df_forecast = df_forecast.drop([4])#drop forth row
df_forecast.head()

4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122,DayNmbr
5,2022-08-18 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,1
6,2022-08-19 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,2
7,2022-08-20 00:00:00,TriCities,0.01,NaN,Prosser,NaN,0.02,NaN,Othello,0.01,3
8,2022-08-21 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,4
9,2022-08-22 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0,5


In [21]:
df_simple = df_forecast[["DATE","RAIN 100148 TRICITIES","RAIN 300033 PROSSER", "RAIN OTHELLO 300122"]]
df_simple.head(10)

4,DATE,RAIN 100148 TRICITIES,RAIN 300033 PROSSER,RAIN OTHELLO 300122
5,2022-08-18 00:00:00,0,0,0
6,2022-08-19 00:00:00,0,0,0
7,2022-08-20 00:00:00,0.01,0.02,0.01
8,2022-08-21 00:00:00,0,0,0
9,2022-08-22 00:00:00,0,0,0
10,2022-08-23 00:00:00,0,0,0
11,2022-08-24 00:00:00,0,0,0
12,2022-08-25 00:00:00,0,0,0
13,2022-08-26 00:00:00,0,0,0
14,2022-08-27 00:00:00,0,0,0


In [22]:
df_simple = df_simple.rename(columns={"RAIN 100148 TRICITIES": "TriCities",
                          "RAIN 300033 PROSSER": "Prosser",
                         "RAIN OTHELLO 300122": "Othello"})
cols_to_check = ['TriCities', 'Prosser', 'Othello']
df_simple[cols_to_check] = df_simple[cols_to_check].replace({'>=':''}, regex=True)
df_simple.head()

4,DATE,TriCities,Prosser,Othello
5,2022-08-18 00:00:00,0.00,0.00,0.00
6,2022-08-19 00:00:00,0.00,0.00,0.00
7,2022-08-20 00:00:00,0.01,0.02,0.01
8,2022-08-21 00:00:00,0.00,0.00,0.00
9,2022-08-22 00:00:00,0.00,0.00,0.00


In [23]:
df_simple.index = pd.to_datetime(df_simple['DATE'], errors = 'coerce')
df_simple= df_simple.drop('DATE',1)
df_simple.head()

C:\Users\theox\AppData\Local\Temp/ipykernel_8692/184375368.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_simple= df_simple.drop('DATE',1)


4,TriCities,Prosser,Othello
DATE,,,
2022-08-18,0.00,0.00,0.00
2022-08-19,0.00,0.00,0.00
2022-08-20,0.01,0.02,0.01
2022-08-21,0.00,0.00,0.00
2022-08-22,0.00,0.00,0.00


In [24]:
grouped_forecast = (df_simple).astype(bool).groupby(df_simple.index.month).sum().reset_index()
grouped_forecast

4,DATE,TriCities,Prosser,Othello
0,8,1,1,1
1,9,0,2,3


In [25]:
df_RainyDays = grouped_forecast.melt(id_vars = ['DATE'], value_name = 'RainyDays')
df_RainyDays.columns = ['Month', 'Station', 'RainyDays']
df_RainyDays

,Month,Station,RainyDays
0,8,TriCities,1
1,9,TriCities,0
2,8,Prosser,1
3,9,Prosser,2
4,8,Othello,1
5,9,Othello,3


In [26]:
#RF_Jul_Othello = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='Othello') & (df_RainyDays['Month'] ==7)])
#RF_Jul_Prosser = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='Prosser') & (df_RainyDays['Month'] ==7)])
#RF_Jul_Tricities = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='TriCities') & (df_RainyDays['Month'] ==7)])
RF_Aug_Othello = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='Othello') & (df_RainyDays['Month'] ==8)])
RF_Aug_Prosser = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='Prosser') & (df_RainyDays['Month'] ==8)])
RF_Aug_Tricities = np.array(df_RainyDays['RainyDays'][(df_RainyDays['Station']=='TriCities') & (df_RainyDays['Month'] ==8)])

In [27]:
RF_Jul_Othello = 0
RF_Jul_Prosser = 0
RF_Jul_Tricities = 0
#RF_Aug_Othello = 0
#RF_Aug_Prosser = 0
#RF_Aug_Tricities = 0

**Calculate diease probablilty for each location based on logistic regression equation**

#### Model 2: logistic regression

- **Othello**

> $lf = 4.960 - 3.084(Y_p) - 0.273(R_{am}) - 0.129(R_{ja})$

In [28]:
lf_Othello = 4.960 - 3.084*0 - 0.273*(RD_Apr_Othello+RD_May_Othello) - 0.129*(RD_Jul_Othello+RD_Aug_Othello+RF_Jul_Othello + RF_Aug_Othello) 
print(lf_Othello)
Prob_Othello = 1 / (1 + np.exp(lf_Othello))
print('Probability of late blight occurence in Othello:')
print(Prob_Othello)

[-2.237]
Probability of late blight occurence in Othello:
[0.90352327]


- **Prosser**

> $lf = 11.513 - 3.894(Y_p) - 0.719(R_{am}) - 0.259(R_{ja})$

> where:

> $Y_p$: late blight outbreak during the preceding year:
$$  
Y_p
\begin{cases}
  \text{late blight absent in previous year/ no} = 0 \\    
  \text{late blight present in previous year/ yes} = 1 
\end{cases}
$$
> $R_{am}$: number of days with rain >= 0.25 mm during April and May

> $R_{ja}$: number of days with rain > = 0.25 during July and August

In [29]:
lf_Prosser = 11.513 - 3.894*0 - 0.719*(RD_Apr_Prosser+ RD_May_Prosser)- 0.259*(RD_Jul_Prosser+ RD_Aug_Prosser+RF_Jul_Prosser + RF_Aug_Prosser)
print(lf_Prosser)
Prob_Prosser= 1 / (1 + np.exp(lf_Prosser))
print('Probability of late blight occurence in Prosser:')
print(Prob_Prosser)

[-4.104]
Probability of late blight occurence in Prosser:
[0.98376152]


- **Tri-Cities**

> $lf = 1.716 - 2.844(Y_p) - 0.094(R_{ja})$

In [30]:
lf_Tricities = 1.716 - 2.844*0 - 0.094*(RD_Jul_Tricities + RD_Aug_Tricities+RF_Jul_Tricities + RF_Aug_Tricities)
print(lf_Tricities)
Prob_Tricities = 1 / (1 + np.exp(lf_Tricities))
print('Probability of late blight occurence in Tricites:')
print(Prob_Tricities)

[1.434]
Probability of late blight occurence in Tricites:
[0.1924762]


**C**

In [200]:
while True:
    try:
        
    # Request user input
        C = input("What is the probability of late blight occurrence in the Columbia Basin of Washington?\n \
                 A) <50% \n \
                 B) >50% \n")
        C = C.upper()
    # If Risk Factor (RF) C is option "A"
        if (C == "A") :
        # Set RF to 0
            C = 0
            break
    # Else, RF is "B" and
        elif (C == "B"):
        # Set it to 30
            C = 4
            break
    #Else, RF is no of the above return warning
        else: 
            print('Please, Enter the valid option!!')
    except ValueError:
        # Break
        break

What is the probability of late blight occurrence in the Columbia Basin of Washington?
                  A) <50% 
                  B) >50% 
 a


**D**

In [201]:
while True:
    try:
    # Request user input
        D = input("What is the date?\n \
                 A) December through April \n \
                 B) May, July - November \n \
                 C) June \n")
        D = D.upper()
    # If Risk Factor (RF) D is option "A"
        if (D == "A") :
        # Set RF to 0
            D = 0
            break
    # Else, if RF is "B" and
        elif (D == "B"):
        # Set it to 1
            D = 1
            break
    # Else, RF is "C" and
        elif (D == "C"):
        # Set it to 3J
            D = 3J
            break
    #Else, RF is no of the above return warning
        else: 
            print('Please, Enter the valid option!!')
    except ValueError:
        # Break
        break

What is the date?
                  A) December through April 
                  B) May, July - November 
                  C) June 
 b


**E**

In [202]:
while True:
    try:
    # Request user input
        E = input("How many rainy days are expected to occur within the next 7 days?\n \
                 A) 0 rainy days expected \n \
                 B) 1 rainy days expected \n \
                 C) 2 rainy days expected \n \
                 D) 3 or more rainy days expected\n")
        E = E.upper()
    # If Risk Factor (RF) E is option "A"
        if (E == "A") :
        # Set RF to 1
            E = 1
            break
    # Else, if RF is "B" and
        elif (E == "B"):
        # Set it to 2
            E = 2
            break
    # Else, if RF is "C" and
        elif (E == "C"):
        # Set it to 4
            E = 4
            break
    # Else, if RF is "C" and
        elif (E == "D"):
        # Set it to 5
            E = 5
            break
    #Else, RF is no of the above return warning
        else: 
            print('Please, Enter the valid option!!')
    except ValueError:
        # Break
        break

How many rainy days are expected to occur within the next 7 days?
                  A) 0 rainy days expected 
                  B) 1 rainy days expected 
                  C) 2 rainy days expected 
                  D) 3 or more rainy days expected
 b


**Calculate Risk factor**

In [ ]:
Risk = A*B*C*D*E
print(Risk)

**Recommendation for the growers**

In [204]:
Risk = Risk.imag if type(Risk) == complex else Risk#change to imagery number is Risk is complex number

if Risk == 0:
    print('Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight on regular bases.')

elif Risk >=1 and Risk <3:
    print('Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')

elif Risk == 3:
    print('Recommendation: Apply fungicides on a 10-14 day schedule through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')

elif Risk == 6: 
    print('Recommendation: Apply fungicides on a 10 day schedule through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')

elif Risk >= 4 and Risk <6:
    print('Recommendation:  Apply late blight fungicide before any rainy period. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')


elif Risk >=8 and Risk <11:
    print('Apply late blight fungicide before any rainy periods and continue for 3 wks. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')

elif Risk == 12:
    print('Recommendation: Apply fungicides on a 7-10 day schedule and before any rainy periods through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')

elif Risk >=15 and Risk <30:
    print('Recommendation: Apply fungicides on a 7 day schedule and before any rainy periods through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')
    
elif Risk >= 30:
    print('Recommendataion: Confirm occurrence of late blight. If present, apply late blight fungicides on a 5 to 7 day schedule and continue until harvest. Avoid over watering and irrigation during and just after rainy, cool and cloudy weather. Harvest during dry weather. Sort out rotten tubers going into storage. Consult literature on recommendations for management of late blight in the field during late season in infected tubers in storage. ')

else:
    print('Oops!!')

Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight on regular bases.


## Graveyard

**Rainy days for April**

In [4]:
os.chdir('C:/Users/sudha.gcupadhaya/Desktop/LateBlight/Forecasts/2021/Monthly')
df_othello = pd.read_csv('Othello.csv')
df_prosser = pd.read_csv('Prosser.csv')
df_tricities = pd.read_csv('Tricities.csv')

In [5]:
df_othello['Station'] = 'Othello'
df_prosser['Station'] = 'Prosser'
df_tricities['Station'] = 'Tricities'

In [6]:
df_AM = pd.concat([df_othello, df_prosser, df_tricities])
df_AM.head()

,Date,Date.1,Min°F,Avg°F,Max°F,Avg1.5m DP°F,Avg1.5m RH%,Avg1.5m LWu.,AvgDir,Avg Speedmph,2m MaxGustmph,2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station,Avg2m Atm.PressinHg
0,2021/04/01,1,31.6,50.1,65.7,28.8,45.7,0.0,E,5.0,17.5,NaN,45.6,47.6,0.0,16.87,0.13,0.19,Othello,NaN
1,2021/04/02,2,33.0,51.7,63.6,32.3,49.9,0.0,E,4.7,14.6,NaN,47.9,49.7,0.0,16.91,0.12,0.16,Othello,NaN
2,2021/04/03,3,38.5,51.0,62.3,31.9,49.6,0.0,E,4.7,12.8,NaN,27.7,49.4,0.0,14.47,0.12,0.16,Othello,NaN
3,2021/04/04,4,38.0,49.7,58.5,31.7,51.5,0.0,E,9.3,23.9,NaN,-1.2,47.6,0.0,13.32,0.13,0.19,Othello,NaN
4,2021/04/05,5,33.5,46.4,58.5,22.0,40.5,0.0,NE,6.7,17.1,NaN,-15.3,44.6,0.0,22.39,0.15,0.21,Othello,NaN


In [7]:
df_AM = df_AM[['Date', 'TotPrecin','Station']]#subset only three columns
print(df_AM.shape)
df_AM.head()

(123, 3)


,Date,TotPrecin,Station
0,2021/04/01,0.0,Othello
1,2021/04/02,0.0,Othello
2,2021/04/03,0.0,Othello
3,2021/04/04,0.0,Othello
4,2021/04/05,0.0,Othello


**Calculate no. of days with rainfall event**

In [26]:
grouped = df_AM[df_AM['TotPrecin'].astype(bool)].groupby(['Station']).size().reset_index(name = 'RainyDays')#calcuate rainy days for each location
grouped

,Station,RainyDays


**Subset no.of rainy days for each location**

In [27]:
RD_April_Othello = np.array(grouped['RainyDays'][(grouped['Station']=='Othello')])
RD_April_Prosser = np.array(grouped['RainyDays'][(grouped['Station']=='Prosser')])
RD_April_Tricities = np.array(grouped['RainyDays'][(grouped['Station']=='Tricities')])